In [1]:
!rm -r dados-abertos/
!git clone https://github.com/SertaoTechnology/dados-abertos.git

rm: cannot remove 'dados-abertos/': No such file or directory
Cloning into 'dados-abertos'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 106 (delta 19), reused 57 (delta 13), pack-reused 0
Receiving objects: 100% (106/106), 2.36 MiB | 7.42 MiB/s, done.
Resolving deltas: 100% (19/19), done.


# Imports

In [2]:
import re

from glob import glob
from collections import OrderedDict

import pandas as pd
import numpy as np

# Carregando Arquivos

In [3]:
file_base_despesas = "dados-abertos/arquivos/raw/paulista/despesas/anual"

file_paths_despesa_total = sorted(
    glob(f"{file_base_despesas}/despesa_total/*.csv")
)

file_paths_despesa_detalhada = sorted(
    glob(f"{file_base_despesas}/despesa_detalhada_unidade/*.csv")
)

file_paths_despesa_credor_empenho = sorted(
    glob(f"{file_base_despesas}/despesa_credor_empenho/*.csv")
)

file_paths_despesa_funcional_programatica = sorted(
    glob(f"{file_base_despesas}/despesa_funcional_programatica/*.csv")
)

# Carrega arquivos de multiplos CSVs como um único DataFrame

In [4]:
def merge_csv(file_paths: list) -> pd.DataFrame:
  dfs = list()
  for file_path in file_paths:
      df = pd.read_csv(file_path, sep=';')
      result = re.search('\w+(\d{4}).csv', file_path)
      if result:
          ano = int(result.groups()[0])
          df['ano'] = ano
      else:
          df['ano'] = pd.NA
      dfs.append(df)

  despesa_total = pd.concat(dfs, axis=0, ignore_index=True)
  return despesa_total

# Preenche as linhas NaN com os valores das linhas anteriores.

In [5]:
class FillNanColumns:

    def __init__(self, *columns):
        self.columns = OrderedDict()
        for column in columns:
            self.columns[column] = None
    
    def __call__(self, row):
        for key, value in self.columns.items():
            if all(row[[key]].notna()):
                self.columns[key] = row[key]
                break
            elif value:
                row[key] = value
        return row.apply(lambda x: x.lower() if isinstance(x, str) else x)

# Converte as colunas de valores BR para float

In [6]:
class ConvertStringToNumeric:

    def __init__(self, *columns):
        self.columns = columns
    
    def __call__(self, row):
        for column in self.columns:
            row[column] = pd.to_numeric(row[column].replace('.', '').replace(',', '.'))
        return row

# Aplicado tratamentos

Os arquivos de **despesa_credor_empenho** não precisam receber o tratamento de colunas NaN e nem de colunas numéricas que estão como string.

In [7]:
despesa_total = merge_csv(file_paths_despesa_total)
despesa_detalhada = merge_csv(file_paths_despesa_detalhada)
despesa_credor_empenho = merge_csv(file_paths_despesa_credor_empenho)
despesa_funcional_programatica = merge_csv(file_paths_despesa_funcional_programatica)

In [8]:
string_columns = FillNanColumns(
    'Categoria Econômica', 'Grupo de despesa', 'Elemento', 'Subelemento'
)

numeric_colmuns = ConvertStringToNumeric(
    'Dotação Inicial', 'Dotação Atualizada', 'Empenhado (E)',
    'Liquidado', 'Pago (P)'
)

despesa_total = despesa_total.apply(
    string_columns, axis=1).dropna().apply(numeric_colmuns, axis=1)

In [9]:
string_columns = FillNanColumns(
    "Órgão", "Unidade", "Categoria Econômica",
    "Grupo de despesa", "Elemento", "Subelemento"
)

numeric_colmuns = ConvertStringToNumeric(
    "Dotação Inicial", "Dotação Atualizada", "Empenhado (P)",
    "Liquidado", "Pago (P)", "P/E"
)

despesa_detalhada = despesa_detalhada.apply(
    string_columns, axis=1
).dropna().apply(numeric_colmuns, axis=1)

In [10]:
string_columns = FillNanColumns(
    "Função", "Subfunção", "Programa",
    "Projeto/Atividade/Operações Especiais", "Ação"
)

numeric_colmuns = ConvertStringToNumeric(
    "Dotação Inicial", "Dotação Atualizada", "Empenhado (E)",
    "Liquidado", "Pago (P)", "P/E"
)

despesa_funcional_programatica = despesa_funcional_programatica.apply(
    string_columns, axis=1
).dropna().apply(numeric_colmuns, axis=1)

# Salva os DataFrames como arquivos CSV

In [11]:
despesa_total.to_csv("despesa_total.csv", sep=";", index=False)
despesa_detalhada.to_csv("despesa_detalhada.csv", sep=";", index=False)
despesa_credor_empenho.to_csv("despesa_credor_empenho.csv", sep=";", index=False)
despesa_funcional_programatica.to_csv("despesa_funcional_programatica.csv", sep=";", index=False)